In [7]:
# Notebook to read in the nexus files and only save the useful stuff to slim the files down
import os
import sys
import tables as tb
import numpy  as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
import re

In [8]:
# Loading in the file
# input_file = "../data/nexus/0nuBB2.next.h5"
# input_file = "../data/nexus/0nuBB2_10bar.next.h5"
input_file = "/Users/mistryk2/Packages/nexus/0nuBB2.next.h5"

h5 = tb.open_file(input_file, 'r')
## Configuration
config = pd.read_hdf(input_file, 'MC/configuration')
display(config)

energy = ""

pressure = config[ config["param_key"] == "/Geometry/XeSphere/pressure"].iloc[0]['param_value']
pressure = [float(num) if '.' in num else int(num) for num in pressure.split() if num.replace('.', '', 1).isdigit()]
pressure = f"{int(pressure[0])}_bar"


eventtype = config[ config["param_key"] == "/nexus/RegisterGenerator"].iloc[0]['param_value']

if (eventtype == "Decay0Interface"):
    eventtype = "bb"
    energy = "2.5_MeV"
else:
    eventtype = "singleeminus"

    # Then also get the energy
    energy = config[ config["param_key"] == "/Generator/SingleParticle/min_energy"].iloc[0]['param_value']
    energy = [float(num) if '.' in num else int(num) for num in energy.split() if num.replace('.', '', 1).isdigit()]
    energy = f"{energy[0]}_MeV"
    

print("Pressure:", pressure) 
print("Energy:", energy) 
print(eventtype)


,param_key,param_value
0,event_type,background
1,num_events,5000
2,saved_events,4686
3,interacting_events,5000
4,/PhysicsList/RegisterPhysics,G4EmStandardPhysics_option4
5,/PhysicsList/RegisterPhysics,G4DecayPhysics
6,/PhysicsList/RegisterPhysics,G4RadioactiveDecayPhysics
7,/PhysicsList/RegisterPhysics,NexusPhysics
8,/PhysicsList/RegisterPhysics,G4StepLimiterPhysics
9,/nexus/RegisterGeometry,XeSphere


Pressure: 1_bar
Energy: 2.5_MeV
singleeminus


In [9]:
## Particles
parts = pd.read_hdf(input_file, 'MC/particles')
print(parts.columns.tolist())
parts = parts.drop(columns=['initial_volume', 'final_volume',
                            'initial_momentum_x', 'initial_momentum_y', 'initial_momentum_z',
                            'final_momentum_x', 'final_momentum_y', 'final_momentum_z'])
display(parts)

['event_id', 'particle_id', 'particle_name', 'primary', 'mother_id', 'initial_x', 'initial_y', 'initial_z', 'initial_t', 'final_x', 'final_y', 'final_z', 'final_t', 'initial_volume', 'final_volume', 'initial_momentum_x', 'initial_momentum_y', 'initial_momentum_z', 'final_momentum_x', 'final_momentum_y', 'final_momentum_z', 'kin_energy', 'length', 'creator_proc', 'final_proc']


,event_id,particle_id,particle_name,primary,mother_id,initial_x,initial_y,initial_z,initial_t,final_x,final_y,final_z,final_t,kin_energy,length,creator_proc,final_proc
0,0,1,e-,1,0,927.853455,-1639.820068,1437.409546,0.000000,2850.097900,-1054.887085,1796.110596,11.562681,2.500000,3.329182e+03,none,eIoni
1,0,104,gamma,0,1,2850.049805,-1054.901855,1796.086548,11.562165,2850.227051,-1055.311768,1796.014160,11.563674,0.038634,4.524398e-01,eBrem,phot
2,0,110,e-,0,104,2850.227051,-1055.311768,1796.014160,11.563674,2850.227051,-1055.311768,1796.014160,11.563674,0.000049,7.220086e-08,phot,msc
3,0,109,e-,0,104,2850.227051,-1055.311768,1796.014160,11.563674,2850.227051,-1055.311768,1796.014160,11.563674,0.000049,7.220086e-08,phot,msc
4,0,108,e-,0,104,2850.227051,-1055.311768,1796.014160,11.563674,2850.227051,-1055.311768,1796.014160,11.563674,0.000774,9.853218e-06,phot,eIoni
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779395,4685,593,e-,0,3,894.642761,-141.783661,-1799.628662,0.473235,894.642761,-141.783600,-1799.628662,0.473253,0.000037,6.407389e-05,eIoni,msc
2779396,4685,592,e-,0,3,894.642761,-141.783661,-1799.628662,0.473235,894.643555,-141.784286,-1799.628418,0.473608,0.000023,1.121595e-03,eIoni,msc
2779397,4685,591,e-,0,3,894.642761,-141.783661,-1799.628662,0.473235,894.639221,-141.786453,-1799.628418,0.473667,0.000401,1.898857e-02,eIoni,eIoni
2779398,4685,590,e-,0,3,894.642761,-141.783661,-1799.628662,0.473235,894.641785,-141.784576,-1799.627808,0.473457,0.000151,5.454436e-03,eIoni,eIoni


In [10]:
## True hits (deposited energy)
hits = pd.read_hdf(input_file, 'MC/hits')
display(hits)
# hits = hits.drop(columns=['label'])

,event_id,x,y,z,time,energy,particle_id,hit_id
0,0,927.907349,-1639.827393,1437.493408,0.000338,0.000066,1,0
1,0,927.914001,-1639.828369,1437.503784,0.000380,0.000005,1,1
2,0,928.021851,-1639.843140,1437.671631,0.001057,0.000022,1,2
3,0,928.164124,-1639.862671,1437.893066,0.001951,0.000301,1,3
4,0,928.272766,-1639.875000,1438.060547,0.002628,0.000039,1,4
...,...,...,...,...,...,...,...,...
80673482,4685,894.639221,-141.786453,-1799.628418,0.473667,0.000288,591,2
80673483,4685,894.641785,-141.784576,-1799.627808,0.473457,0.000151,590,0
80673484,4685,904.463745,-155.291824,-1770.876953,0.337490,0.000316,2,0
80673485,4685,904.468506,-155.294540,-1770.888306,0.337963,0.000523,2,1


In [11]:
# Open the HDF5 file in write mode
with pd.HDFStore(f"../data/nexus/1bar/models/xesphere_{eventtype}_{pressure}_{energy}_low_step.h5", mode='w', complevel=5, complib='zlib') as store:
    # Write each DataFrame to the file with a unique key
    store.put('config', config, format='table')
    store.put('parts',parts, format='table')
    store.put('hits',hits, format='table')

In [12]:
# load back in to check

hits2 = pd.read_hdf(f"../data/nexus/1bar/models/xesphere_{eventtype}_{pressure}_{energy}_low_step.h5", 'hits')
display(hits2)

,event_id,x,y,z,time,energy,particle_id,hit_id
0,0,927.907349,-1639.827393,1437.493408,0.000338,0.000066,1,0
1,0,927.914001,-1639.828369,1437.503784,0.000380,0.000005,1,1
2,0,928.021851,-1639.843140,1437.671631,0.001057,0.000022,1,2
3,0,928.164124,-1639.862671,1437.893066,0.001951,0.000301,1,3
4,0,928.272766,-1639.875000,1438.060547,0.002628,0.000039,1,4
...,...,...,...,...,...,...,...,...
80673482,4685,894.639221,-141.786453,-1799.628418,0.473667,0.000288,591,2
80673483,4685,894.641785,-141.784576,-1799.627808,0.473457,0.000151,590,0
80673484,4685,904.463745,-155.291824,-1770.876953,0.337490,0.000316,2,0
80673485,4685,904.468506,-155.294540,-1770.888306,0.337963,0.000523,2,1


In [24]:
event_hits = hits2[hits2.event_id == 2]
event_hits["n"] = round(event_hits["energy"]/22e-6)
display(event_hits)
print(event_hits.energy.sum())
print(0.002081/25e-6)
print(event_hits.n.sum())

/var/folders/tr/r0bv1gl52_vb7v8bswkb_z0nb301gc/T/ipykernel_86304/1087483978.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_hits["n"] = round(event_hits["energy"]/22e-6)


,event_id,x,y,z,time,energy,particle_id,hit_id,n
33392,2,444.381470,520.242310,562.902344,0.001275,0.000023,1,0,1.0
33393,2,444.713928,520.098145,563.055298,0.002607,0.000634,1,1,29.0
33394,2,444.882935,520.025818,563.134155,0.003284,0.000031,1,2,1.0
33395,2,445.430725,519.791321,563.388794,0.005477,0.000038,1,3,2.0
33396,2,445.516876,519.755249,563.424500,0.005816,0.000019,1,4,1.0
...,...,...,...,...,...,...,...,...,...
52000,2,482.166718,495.101440,585.306946,0.174054,0.002081,2,1,95.0
52001,2,482.172791,495.100311,585.303101,0.174398,0.000335,2,2,15.0
52002,2,482.173676,495.097870,585.298584,0.174682,0.000138,2,3,6.0
52003,2,482.172852,495.096008,585.294739,0.174940,0.000382,2,4,17.0


2.5
83.24
113430.0


1.8187577
